In [1]:
import pandas as pd
import numpy as np
import pydicom
import cv2

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 필요한 만큼만 메모리를 사용하도록 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        # 특정 GPU에 연산을 할당
        tf.config.set_visible_devices(gpus[0], 'GPU')
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
train_df = pd.read_pickle('train_data.pkl')

In [4]:
val_df= pd.read_pickle('val_data.pkl')

In [5]:
# 이미지 전처리


from tensorflow.keras.utils import Sequence
import numpy as np
import pydicom
import cv2

class DicomDataGenerator(Sequence):
    def __init__(self, dataframe, x_col, y_col, batch_size, target_size, shuffle=True):
        self.dataframe = dataframe
        self.x_col = x_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch = self.dataframe.iloc[index*self.batch_size:(index+1)*self.batch_size]
        x, y = self.__data_generation(batch)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.dataframe))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch):
        images = []
        labels = []

        for _, row in batch.iterrows():
            dicom_path = row[self.x_col]
            dicom = pydicom.dcmread(dicom_path)
            image = dicom.pixel_array
            image = cv2.resize(image, self.target_size)
            image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
            image = image.astype('float32') / 255.0
            image = np.expand_dims(image, axis=-1)
            images.append(image)
            labels.append(row[self.y_col])
        
        x = np.array(images)
        y = tf.keras.utils.to_categorical(labels, num_classes=3)
        return x, y

# 사용 예시
# train_df와 val_df는 DataFrame 형태로, 'img_file_path'와 'category' 컬럼이 있다고 가정
train_df['category'] = train_df['category'].astype(int)
val_df['category'] = val_df['category'].astype(int)

train_generator = DicomDataGenerator(
    dataframe=train_df,
    x_col='img_file_path',
    y_col='category',
    batch_size=32,
    target_size=(224, 224),
    shuffle=True
)

val_generator = DicomDataGenerator(
    dataframe=val_df,
    x_col='img_file_path',
    y_col='category',
    batch_size=32,
    target_size=(224, 224),
    shuffle=False
)

In [6]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# CNN 모델 정의
cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten()
])

# CNN 모델 출력 특징 추출
def extract_features(generator, model):
    features = []
    labels = []
    for x_batch, y_batch in generator:
        feature = model.predict(x_batch)
        features.append(feature)
        labels.append(y_batch)
    return np.concatenate(features), np.concatenate(labels)

In [9]:
import os

# 특징을 디스크에 저장하는 함수
def save_features_to_disk(generator, model, save_dir, prefix):
    os.makedirs(save_dir, exist_ok=True)
    for i, (x_batch, y_batch) in enumerate(generator):
        feature = model.predict(x_batch)
        feature_path = os.path.join(save_dir, f"{prefix}_features_{i}.npy")
        label_path = os.path.join(save_dir, f"{prefix}_labels_{i}.npy")
        np.save(feature_path, feature)
        np.save(label_path, y_batch)
        print(f"Saved {feature_path} and {label_path}")


# 특징 추출 및 저장
save_features_to_disk(train_generator, cnn, 'train_features', 'train')
save_features_to_disk(val_generator, cnn, 'val_features', 'val')

1/1 [==============================] - 0s 18ms/step
Saved train_features\train_features_0.npy and train_features\train_labels_0.npy
1/1 [==============================] - 0s 17ms/step
Saved train_features\train_features_1.npy and train_features\train_labels_1.npy
1/1 [==============================] - 0s 16ms/step
Saved train_features\train_features_2.npy and train_features\train_labels_2.npy
1/1 [==============================] - 0s 15ms/step
Saved train_features\train_features_3.npy and train_features\train_labels_3.npy
1/1 [==============================] - 0s 16ms/step
Saved train_features\train_features_4.npy and train_features\train_labels_4.npy
1/1 [==============================] - 0s 17ms/step
Saved train_features\train_features_5.npy and train_features\train_labels_5.npy
1/1 [==============================] - 0s 14ms/step
Saved train_features\train_features_6.npy and train_features\train_labels_6.npy
1/1 [==============================] - 0s 15ms/step
Saved train_features\tra

In [13]:
# 저장된 파일 로드하는 함수
def load_features_from_disk(save_dir, prefix):
    features = []
    labels = []
    saved_files = os.listdir(save_dir)  # 디렉토리 내 파일 리스트

    # 파일 목록 출력
    print(f"Files in {save_dir}: {saved_files}")

    # 숫자 기준으로 정렬된 파일 목록 생성
    feature_files = sorted(
        [f for f in saved_files if f.startswith(f'{prefix}_features') and f.endswith('.npy')],
        key=lambda x: int(''.join(filter(str.isdigit, x)))
    )
    label_files = sorted(
        [f for f in saved_files if f.startswith(f'{prefix}_labels') and f.endswith('.npy')],
        key=lambda x: int(''.join(filter(str.isdigit, x)))
    )

    if not feature_files or not label_files:
        raise ValueError("No files found to load.")

    # 파일 쌍을 처리하여 features와 labels를 로드
    for feature_file in feature_files:
        feature_path = os.path.join(save_dir, feature_file)
        label_file = feature_file.replace('_features_', '_labels_')  # Assuming labels are named similarly
        label_path = os.path.join(save_dir, label_file)

        if label_file not in label_files:
            raise ValueError(f"Matching label file not found for {feature_file}.")

        features.append(np.load(feature_path))
        labels.append(np.load(label_path))
        print(f"Loaded {feature_path} and {label_path}")

    return np.concatenate(features), np.concatenate(labels)

# 저장된 파일 로드
train_features, train_labels = load_features_from_disk('train_features', 'train')
val_features, val_labels = load_features_from_disk('val_features', 'val')

Files in train_features: ['train_features_0.npy', 'train_features_1.npy', 'train_features_10.npy', 'train_features_100.npy', 'train_features_1000.npy', 'train_features_1001.npy', 'train_features_1002.npy', 'train_features_1003.npy', 'train_features_1004.npy', 'train_features_1005.npy', 'train_features_1006.npy', 'train_features_1007.npy', 'train_features_1008.npy', 'train_features_1009.npy', 'train_features_101.npy', 'train_features_1010.npy', 'train_features_1011.npy', 'train_features_1012.npy', 'train_features_1013.npy', 'train_features_1014.npy', 'train_features_1015.npy', 'train_features_1016.npy', 'train_features_1017.npy', 'train_features_1018.npy', 'train_features_1019.npy', 'train_features_102.npy', 'train_features_1020.npy', 'train_features_1021.npy', 'train_features_1022.npy', 'train_features_1023.npy', 'train_features_1024.npy', 'train_features_1025.npy', 'train_features_1026.npy', 'train_features_1027.npy', 'train_features_1028.npy', 'train_features_1029.npy', 'train_featur

In [ ]:
'''
# 저장된 파일 로드하는 함수
def load_features_from_disk(save_dir, prefix):
    features = []
    labels = []
    for feature_file in sorted(os.listdir(save_dir)):
        if feature_file.startswith(prefix) and feature_file.endswith('_features.npy'):
            feature_path = os.path.join(save_dir, feature_file)
            label_path = feature_path.replace('_features.npy', '_labels.npy')
            features.append(np.load(feature_path))
            labels.append(np.load(label_path))
            print(f"Loaded {feature_path} and {label_path}")
    if not features or not labels:
        raise ValueError("No files found to load.")
    return np.concatenate(features), np.concatenate(labels)



# 저장된 파일 로드
train_features, train_labels = load_features_from_disk('train_features', 'train')
val_features, val_labels = load_features_from_disk('val_features', 'val')
'''

In [8]:
'''
# 특징 추출
train_features, train_labels = extract_features(train_generator, cnn)
val_features, val_labels = extract_features(val_generator, cnn)
'''

'\n# 특징 추출\ntrain_features, train_labels = extract_features(train_generator, cnn)\nval_features, val_labels = extract_features(val_generator, cnn)\n'

In [14]:
# 차원을 평탄화
train_features = train_features.reshape(train_features.shape[0], -1)
val_features = val_features.reshape(val_features.shape[0], -1)

In [15]:
# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, np.argmax(train_labels, axis=1))

RandomForestClassifier(random_state=42)

In [16]:
# 예측 및 성능 평가
val_predictions = rf_model.predict(val_features)
val_accuracy = accuracy_score(np.argmax(val_labels, axis=1), val_predictions)

print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.7630550986842105


In [ ]:
'''
cnn + rdf(no generator)
Validation Accuracy: 0.7630550986842105
'''

In [17]:
import joblib

# 모델 저장
joblib.dump(rf_model, 'rdf_cnn_no_generator.pkl')

['rdf_cnn_no_generator.pkl']